<a href="https://colab.research.google.com/github/jimmyland22/CMPE260/blob/main/CMPE260_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -L https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2 | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!curl -L https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2 | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3737    0  3737    0     0  38927      0 --:--:-- --:--:-- --:--:-- 38927
100  503k  100  503k    0     0  2044k      0 --:--:-- --:--:-- --:--:-- 3645k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3745    0  3745    0     0  42556      0 --:--:-- --:--:-- --:--:-- 42556
100  406k  100  406k    0     0  1715k      0 --:--:-- --:--:-- --:--:-- 1715k


In [2]:
import gym
import numpy as np
import pandas as pd
import sys
import talib
from gym import spaces
from gym.envs.registration import register
from gym.utils import seeding
from sklearn.preprocessing import scale

sys.modules[__name__]

<module '__main__'>

In [3]:
class DataSource:
    def __init__(self, trading_days=252, ticker="AAPL", normalize=True):
        self.trading_days = trading_days
        self.ticker = ticker
        self.normalize = normalize
        self.data = self.load_data()
        self.preprocess_data()
        self.min_values = self.data.min()
        self.max_values = self.data.max()
        self.step = 0
        self.offset = None

    def load_data(self):
        print("Loading data for {}".format(self.ticker))
        idx = pd.IndexSlice
        with pd.HDFStore("data/assets.h5") as store:
            df = (store["quandl/wiki/prices"]
                  .loc[idx[:, self.ticker], ["adj_close", "adj_volume", "adj_low", "adj_high"]]
                  .dropna()
                  .sort_index())
        df.columns = ["close", "volume", "low", "high"]
        print("Loaded data for {}".format(self.ticker))
        return df

    def preprocess_data(self):
        self.data["returns"] = self.data.close.pct_change()
        self.data["ret_2"] = self.data.close.pct_change(2)
        self.data["ret_5"] = self.data.close.pct_change(5)
        self.data["ret_10"] = self.data.close.pct_change(10)
        self.data["ret_21"] = self.data.close.pct_change(21)
        self.data["rsi"] = talib.STOCHRSI(self.data.close)[1]
        self.data["macd"] = talib.MACD(self.data.close)[1]
        self.data["atr"] = talib.ATR(self.data.high, self.data.low, self.data.close)
        slowk, slowd = talib.STOCH(self.data.high, self.data.low, self.data.close)
        self.data["stoch"] = slowd - slowk
        self.data["atr"] = talib.ATR(self.data.high, self.data.low, self.data.close)
        self.data["ultosc"] = talib.ULTOSC(self.data.high, self.data.low, self.data.close)
        self.data = (self.data
                     .replace((np.inf, -np.inf), np.nan)
                     .drop(["high", "low", "close", "volume"], axis=1)
                     .dropna())
        r = self.data.returns.copy()
        if self.normalize:
            self.data = pd.DataFrame(scale(self.data), columns=self.data.columns, index=self.data.index)
        features = self.data.columns.drop("returns")
        self.data["returns"] = r
        self.data = self.data.loc[:, ["returns"] + list(features)]
        print(self.data.info())

    def reset(self):
        high = len(self.data.index) - self.trading_days
        self.offset = np.random.randint(low=0, high=high)
        self.step = 0

    def take_step(self):
        obs = self.data.iloc[self.offset + self.step].values
        self.step += 1
        done = self.step > self.trading_days
        return obs, done

In [4]:
class TradingSimulator:
    def __init__(self, steps, trading_cost_bps, time_cost_bps):
        self.steps = steps
        self.trading_cost_bps = trading_cost_bps
        self.time_cost_bps = time_cost_bps
        self.step = 0
        self.actions = np.zeros(self.steps)
        self.navs = np.ones(self.steps)
        self.market_navs = np.ones(self.steps)
        self.strategy_returns = np.ones(self.steps)
        self.positions = np.zeros(self.steps)
        self.costs = np.zeros(self.steps)
        self.trades = np.zeros(self.steps)
        self.market_returns = np.zeros(self.steps)

    def reset(self):
        self.step = 0
        self.actions.fill(0)
        self.navs.fill(1)
        self.market_navs.fill(1)
        self.strategy_returns.fill(0)
        self.positions.fill(0)
        self.costs.fill(0)
        self.trades.fill(0)
        self.market_returns.fill(0)

    def take_step(self, action, market_return):
        start_position = self.positions[max(0, self.step - 1)]
        start_nav = self.navs[max(0, self.step - 1)]
        start_market_nav = self.market_navs[max(0, self.step - 1)]
        self.market_returns[self.step] = market_return
        self.actions[self.step] = action
        end_position = action - 1
        n_trades = end_position - start_position
        self.positions[self.step] = end_position
        self.trades[self.step] = n_trades
        trade_costs = abs(n_trades) * self.trading_cost_bps
        time_cost = 0 if n_trades else self.time_cost_bps
        self.costs[self.step] = trade_costs + time_cost
        reward = start_position * market_return - self.costs[self.step]
        self.strategy_returns[self.step] = reward
        if self.step != 0:
            self.navs[self.step] = start_nav * (1 + self.strategy_returns[self.step])
            self.market_navs[self.step] = start_market_nav * (1 + self.market_returns[self.step])
        info = {"reward": reward,
                "nav": self.navs[self.step],
                "costs": self.costs[self.step]}
        self.step += 1
        return reward, info

    def result(self):
        return pd.DataFrame({"action": self.actions,
                             "nav": self.navs,
                             "market_nav": self.market_navs,
                             "market_return": self.market_returns,
                             "strategy_return": self.strategy_returns,
                             "position": self.positions,
                             "cost": self.costs,
                             "trade": self.trades})

In [5]:
class TradingEnvironment(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(self, trading_days=252, trading_cost_bps=1e-3, time_cost_bps=1e-4, ticker="AAPL"):
        self.trading_days = trading_days
        self.trading_cost_bps = trading_cost_bps
        self.time_cost_bps = time_cost_bps
        self.ticker = ticker
        self.data_source = DataSource(self.trading_days, ticker)
        self.simulator = TradingSimulator(self.trading_days, self.trading_cost_bps, self.time_cost_bps)
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(self.data_source.min_values, self.data_source.max_values)
        self.reset()
        self.np_random = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        assert self.action_space.contains(action), "{} {} invalid".format(action, type(action))
        observation, done = self.data_source.take_step()
        reward, info = self.simulator.take_step(action, observation[0])
        return observation, reward, done, info

    def reset(self):
        self.data_source.reset()
        self.simulator.reset()
        return self.data_source.take_step()[0]

    def render(self, mode="human"):
        pass

In [6]:
!rm -rf *
!mkdir data
!mkdir temp
!git clone https://github.com/jimmyland22/CMPE260.git
for i in range(1, 7):
    zip = "/content/CMPE260/data/part_{}.zip".format(i)
    !unzip $zip -d temp
!cat temp/*csv > data/wiki_prices.csv
!rm -rf CMPE260/
!rm -rf temp/

Cloning into 'CMPE260'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 1), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.
Checking out files: 100% (10/10), done.
Archive:  /content/CMPE260/data/part_1.zip
  inflating: temp/xaa.csv            
Archive:  /content/CMPE260/data/part_2.zip
  inflating: temp/xab.csv            
Archive:  /content/CMPE260/data/part_3.zip
  inflating: temp/xac.csv            
Archive:  /content/CMPE260/data/part_4.zip
  inflating: temp/xad.csv            
Archive:  /content/CMPE260/data/part_5.zip
  inflating: temp/xae.csv            
Archive:  /content/CMPE260/data/part_6.zip
  inflating: temp/xaf.csv            


In [7]:
pd.set_option("display.expand_frame_repr", False)

df = pd.read_csv("data/wiki_prices.csv",
                 parse_dates=["date"],
                 index_col=["date", "ticker"],
                 infer_datetime_format=True).sort_index()

with pd.HDFStore("data/assets.h5") as store:
    store.put("quandl/wiki/prices", df)

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   open         float64
 1   high         float64
 2   low          float64
 3   close        float64
 4   volume       float64
 5   ex-dividend  float64
 6   split_ratio  float64
 7   adj_open     float64
 8   adj_high     float64
 9   adj_low      float64
 10  adj_close    float64
 11  adj_volume   float64
dtypes: float64(12)
memory usage: 1.4+ GB
None


open    high     low   close   volume  ex-dividend  split_ratio  adj_open  adj_high   adj_low  adj_close  adj_volume
date       ticker                                                                                                                        
1962-01-02 ARNC     65.56   65.75   65.38   65.38   5600.0          0.0          1.0  3.458163  3.468185  3.448668   3.448668   44800.000
           BA       50.88   50.88   50.00   50.00  11595.0          0.0          1.0  0.887160  0.887160  0.871816   0.871816  352198.125
           CAT      38.50   38.87   38.12   38.50  13600.0          0.0          1.0  1.578370  1.593539  1.562791   1.578370  163200.000
           DD      241.50  244.25  241.50  241.50   2000.0          0.0          1.0  4.718414  4.772144  4.718414   4.718414   36000.000
           DIS      37.25   38.50   37.25   37.25   2098.0          0.0          1.0  0.141259  0.145999  0.141259   0.141259  408858.240

In [8]:
trading_days = 252
trading_cost_bps = 0
time_cost_bps = 0
ticker = "GOOG"

In [9]:
register(id="trading-v0",
         entry_point="__main__:TradingEnvironment",
         max_episode_steps=trading_days)

In [10]:
trading_environment = gym.make("trading-v0",
                               trading_days=trading_days,
                               trading_cost_bps=trading_cost_bps,
                               time_cost_bps=time_cost_bps,
                               ticker=ticker)

Loading data for GOOG
Loaded data for GOOG
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 974 entries, (Timestamp('2014-05-14 00:00:00'), 'GOOG') to (Timestamp('2018-03-27 00:00:00'), 'GOOG')
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   returns  974 non-null    float64
 1   ret_2    974 non-null    float64
 2   ret_5    974 non-null    float64
 3   ret_10   974 non-null    float64
 4   ret_21   974 non-null    float64
 5   rsi      974 non-null    float64
 6   macd     974 non-null    float64
 7   atr      974 non-null    float64
 8   stoch    974 non-null    float64
 9   ultosc   974 non-null    float64
dtypes: float64(10)
memory usage: 1016.5+ KB
None


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [11]:
trading_environment.observation_space

Box(-4.426818370819092, 9.67429256439209, (10,), float32)

In [12]:
trading_environment.action_space

Discrete(3)

In [13]:
trading_environment.spec.max_episode_steps

252

In [14]:
trading_environment.reset()
trading_environment.step(2)

(array([ 0.01135562,  1.32277149,  1.32954803,  1.33426847,  0.29528349,
         1.36839818, -1.22513252, -0.23487574, -1.2071162 ,  0.71946664]),
 0.0,
 False,
 {'costs': 0.0, 'nav': 1.0, 'reward': 0.0})